## capstone project notebook

In [7]:
!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

--2020-12-05 15:02:01--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23101 (23K) [text/csv]
Saving to: ‘loan_train.csv’

loan_train.csv      100%[===================>]  22.56K  --.-KB/s    in 0.07s   

2020-12-05 15:02:02 (304 KB/s) - ‘loan_train.csv’ saved [23101/23101]



### Steps:
1. Create our models, training them with the train-dataset (loan_train.csv)
2. Meassure the accuracy of each of the build model

# 1. Create models

### Process/Clean data our train dataset

In [81]:
import random 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs 
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

# Load Dataset:
import pandas as pd
df = pd.read_csv("loan_train.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,4,4,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,6,6,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male


In [82]:
# Pre-processing of train dateset
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df['dayofweek'] = df['effective_date'].dt.dayofweek
df['weekend']= df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
Feature = df[['Principal','terms','age','Gender','weekend']]
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()

,Principal,terms,age,Gender,weekend,Bechalor,High School or Below,college
0,1000,30,45,0,0,0,1,0
1,1000,30,33,1,0,1,0,0
2,1000,15,27,0,0,0,0,1
3,1000,30,28,1,1,0,0,1
4,1000,30,29,0,1,0,0,1


In [83]:
# lets rename variables X, Y
X = Feature
# Normalize data
X = preprocessing.StandardScaler().fit(X).transform(X)
y = df['loan_status'].values
X

array([[ 0.51578458,  0.92071769,  2.33152555, ..., -0.38170062,
         1.13639374, -0.86968108],
       [ 0.51578458,  0.92071769,  0.34170148, ...,  2.61985426,
        -0.87997669, -0.86968108],
       [ 0.51578458, -0.95911111, -0.65321055, ..., -0.38170062,
        -0.87997669,  1.14984679],
       ...,
       [-1.31458942, -0.95911111,  1.33661351, ..., -0.38170062,
        -0.87997669,  1.14984679],
       [ 0.51578458,  0.92071769, -0.48739188, ..., -0.38170062,
        -0.87997669,  1.14984679],
       [ 0.51578458,  0.92071769, -0.81902922, ..., -0.38170062,
        -0.87997669,  1.14984679]])

### Split train dataset in train and test (not needed actually)

In [84]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# usual split in test/train sub-datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
print ('data to train:', X_train.shape,  y_train.shape)
print ('data to test:', X_test.shape,  y_test.shape)

data to train: (276, 8) (276,)
data to test: (70, 8) (70,)


### K-Nearest model


#### K=1...15

In [120]:
Ks=15
kNN_model={}
for n in range(1,Ks):    
    kNN_model[n] = KNeighborsClassifier(n_neighbors=n).fit(X_train,y_train)

### build decision-trees model

In [121]:
from sklearn.tree import DecisionTreeClassifier
DT_model = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
DT_model.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

### build Support Vector Machine model


In [122]:
from sklearn import svm
SVM_model = svm.SVC()
SVM_model.fit(X_train, y_train)

SVC()

### build Logistic Regression model


In [123]:
from sklearn.linear_model import LogisticRegression
LR_model = LogisticRegression(C=0.01).fit(X_train,y_train)

# 2. Study accuracy of our previously built models using the test dataset:

In [124]:
# from sklearn.metrics import jaccard_similarity_score   ----> seems deprecated
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

!wget -O loan_test.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv
    
# we want to meassure the accuracy of our models with a test dataset. Until now we were working with loan_train.csv
test_df = pd.read_csv('loan_test.csv')
test_df.head()

--2020-12-07 20:02:49--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_test.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3642 (3.6K) [text/csv]
Saving to: ‘loan_test.csv’

loan_test.csv       100%[===================>]   3.56K  --.-KB/s    in 0s      

2020-12-07 20:02:49 (80.3 MB/s) - ‘loan_test.csv’ saved [3642/3642]



,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,1,1,PAIDOFF,1000,30,9/8/2016,10/7/2016,50,Bechalor,female
1,5,5,PAIDOFF,300,7,9/9/2016,9/15/2016,35,Master or Above,male
2,21,21,PAIDOFF,1000,30,9/10/2016,10/9/2016,43,High School or Below,female
3,24,24,PAIDOFF,1000,30,9/10/2016,10/9/2016,26,college,male
4,35,35,PAIDOFF,800,15,9/11/2016,9/25/2016,29,Bechalor,male


In [125]:
# Same transformations as before:
# - convert to datetimes
test_df['due_date'] = pd.to_datetime(test_df['due_date'])
test_df['effective_date'] = pd.to_datetime(test_df['effective_date'])
# add dayofweek and weekend
test_df['dayofweek'] = test_df['effective_date'].dt.dayofweek
test_df['weekend'] = test_df['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)
# convert gender to binary
test_df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
test_Feature = test_df[['Principal','terms','age','Gender','weekend']]
# convert all the type of educations to binary columns
test_Feature = pd.concat([test_Feature,pd.get_dummies(test_df['education'])], axis=1)
test_Feature.drop(['Master or Above'], axis = 1,inplace=True)
test_Feature.head()

,Principal,terms,age,Gender,weekend,Bechalor,High School or Below,college
0,1000,30,50,1,0,1,0,0
1,300,7,35,0,1,0,0,0
2,1000,30,43,1,1,0,1,0
3,1000,30,26,0,1,0,0,1
4,800,15,29,0,1,1,0,0


In [126]:
# master or above are clearly too busy to pay loans
test_X = preprocessing.StandardScaler().fit(test_Feature).transform(test_Feature)
test_X[0:5]

array([[ 0.49362588,  0.92844966,  3.05981865,  1.97714211, -1.30384048,
         2.39791576, -0.79772404, -0.86135677],
       [-3.56269116, -1.70427745,  0.53336288, -0.50578054,  0.76696499,
        -0.41702883, -0.79772404, -0.86135677],
       [ 0.49362588,  0.92844966,  1.88080596,  1.97714211,  0.76696499,
        -0.41702883,  1.25356634, -0.86135677],
       [ 0.49362588,  0.92844966, -0.98251057, -0.50578054,  0.76696499,
        -0.41702883, -0.79772404,  1.16095912],
       [-0.66532184, -0.78854628, -0.47721942, -0.50578054,  0.76696499,
         2.39791576, -0.79772404, -0.86135677]])

### Accuracy for our built K-Nearest model

In [137]:
for n in range(1,15):
    knn_yhat = kNN_model[n].predict(test_X)
    print("KNN FOR K=" + str(n))
    print("\tJaccard index:\t %.2f" % jaccard_score(test_y, knn_yhat, average='weighted'))    
    print("\tF1-score:\t %.2f" % f1_score(test_y, knn_yhat, average='weighted') )

KNN FOR K=1
	Jaccard index:	 0.55
	F1-score:	 0.69
KNN FOR K=2
	Jaccard index:	 0.49
	F1-score:	 0.65
KNN FOR K=3
	Jaccard index:	 0.56
	F1-score:	 0.70
KNN FOR K=4
	Jaccard index:	 0.51
	F1-score:	 0.66
KNN FOR K=5
	Jaccard index:	 0.56
	F1-score:	 0.69
KNN FOR K=6
	Jaccard index:	 0.55
	F1-score:	 0.69
KNN FOR K=7
	Jaccard index:	 0.51
	F1-score:	 0.63
KNN FOR K=8
	Jaccard index:	 0.59
	F1-score:	 0.73
KNN FOR K=9
	Jaccard index:	 0.55
	F1-score:	 0.67
KNN FOR K=10
	Jaccard index:	 0.56
	F1-score:	 0.70
KNN FOR K=11
	Jaccard index:	 0.55
	F1-score:	 0.67
KNN FOR K=12
	Jaccard index:	 0.52
	F1-score:	 0.65
KNN FOR K=13
	Jaccard index:	 0.53
	F1-score:	 0.65
KNN FOR K=14
	Jaccard index:	 0.53
	F1-score:	 0.66


**====> We can observe that the best K is for K=8:**
    
	Jaccard index:	 0.59 
    
    F1-score:	     0.73 
    

### Accuracy for our built decission-trees model

In [130]:
DT_yhat = DT_model.predict(test_X)
print("DT Jaccard index: %.2f" % jaccard_score(test_y, DT_yhat, average='weighted'))
print("DT F1-score: %.2f" % f1_score(test_y, DT_yhat, average='weighted') )

DT Jaccard index: 0.59
DT F1-score: 0.74


### Accuracy for our built Suppor-Vector-Machine model

In [131]:
SVM_yhat = SVM_model.predict(test_X)
print("SVM Jaccard index: %.2f" % jaccard_score(test_y, SVM_yhat, average='weighted'))
print("SVM F1-score: %.2f" % f1_score(test_y, SVM_yhat, average='weighted') )

SVM Jaccard index: 0.65
SVM F1-score: 0.76


### Accuracy for our built Logistic regression model

In [132]:
LR_yhat = LR_model.predict(test_X)
LR_yhat_prob = LR_model.predict_proba(test_X)
print("LR Jaccard index: %.2f" % jaccard_score(test_y, LR_yhat, average='weighted'))
print("LR F1-score: %.2f" % f1_score(test_y, LR_yhat, average='weighted') )
print("LR LogLoss: %.2f" % log_loss(test_y, LR_yhat_prob))

LR Jaccard index: 0.55
LR F1-score: 0.63
LR LogLoss: 0.52


**===> We can observe that we have better results with Support Vector Machine**